# House Prices: Advanced Regression Techniques, Part 1
## Load, explore, and clean the data

In [1]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from IPython.display import display
pd.options.display.precision = 4
mpl.rcParams['font.family'] = 'Lato'
mpl.rcParams['font.weight'] = 700
sns.set(font='Lato', font_scale=1)
sns.set()
np.random.seed(42)

https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data

Ask a home buyer to describe their dream house, and they probably won't begin with the height of the basement ceiling or the proximity to an east-west railroad. But this playground competition's dataset proves that much more influences price negotiations than the number of bedrooms or a white-picket fence.

With 79 explanatory variables describing (almost) every aspect of residential homes in Ames, Iowa, this competition challenges you to predict the final price of each home.

## Load the Data

The data set is split across three tables:

- `listing_training`
- `zoning`
- `sale`

Load the data from these tables:
1. connect to the database using `psycopg2`
1. use a `psycopg2.extras.RealDictCursor` so that you get the columns names returned with your query
1. pull the data using a `SELECT` `JOIN` statement 
1. you should be able to join all tables using `id`
1. load the result into a `pandas.DataFrame`
1. use `id` as the index for your DataFrame

In [2]:
import psycopg2 as pg2
import psycopg2.extras as pgex

this_host='michaelgfrantz.com'
this_user='postgres'
this_password='dsism4'

conn = pg2.connect(host = this_host, 
                        user = this_user,
                        password = this_password)
curs = conn.cursor(cursor_factory=pgex.RealDictCursor)
#execute SQL query
query = """  select * 
            from listing_training l
            join zoning z on l.id = z.id
            join sale s on l.id = s.id;
"""
curs.execute(query)
results = curs.fetchall()
conn.close()
houseprice_df = pd.DataFrame(results)
houseprice_df.set_index('id',drop=True, inplace = True)
houseprice_df.shape

OperationalError: could not connect to server: Connection refused
	Is the server running on host "michaelgfrantz.com" (162.243.140.117) and accepting
	TCP/IP connections on port 5432?


In [ ]:
houseprice_df.head()

### What are the names of the columns?

In [ ]:
houseprice_df.columns.values

### Find all the columns with over 500 null values

Use Python to find and drop these columns.

In [ ]:
sum(houseprice_df['alley'].isnull())

In [ ]:
column_500null = []
houseprice_df_cleaning = houseprice_df.copy()
for col in houseprice_df.columns:
    if sum(houseprice_df[col].isnull()) > 500:
        houseprice_df_cleaning.drop(col,axis =1, inplace=True)
        column_500null.append(col)
column_500null


### How many columns of each data type are there?

In [ ]:
data_type={}
for col in houseprice_df_cleaning.columns:
    key = houseprice_df_cleaning[col].dtype
    if key in data_type:
        data_type[key] += 1
    else: 
        data_type[key] = 1
data_type        

In [ ]:
houseprice_df_cleaning.get_dtype_counts()

### Create a Meta-Information DataFrame

1. call it `feature_info_df`
1. make an empty list called `feature_info`
1. for each column make a dictionary:  
   `{'feature':<column_name>,  
    'n_unique': <number_of_unique_elements>,  
    'datatype': <datatype_of_the_feature>}`
1. append the dictionary to the list `feature_info`
1. use the list of dictionaries to create a new dataframe

In [ ]:
feature_info_df = houseprice_df_cleaning.copy()
feature_info = []
for col in feature_info_df.columns:
    info = { 
        'feature': col,
        'n_unique': len(set(feature_info_df[col])),
        'datatype': feature_info_df[col].dtype
    }
    feature_info.append(info)
feature_info_df = pd.DataFrame(feature_info)  
feature_info_df.head()

### Display the meta information for all of the integers columns

In [ ]:
feature_info_int_df = feature_info_df[feature_info_df['datatype']=='int64']
feature_info_int_df.head()

### Use the meta-info dataframe to find all the integer features with 15 or less unique values

Make a list of these columns. We will change the type of these features to 'object' in our original dataframe.

In [ ]:
object_list = []
for i in range(0,len(feature_info_int_df)):
   
    if feature_info_int_df['n_unique'].iloc[i] <15:
        object_list.append(feature_info_int_df['feature'].iloc[i])
object_list        

### Change the type of these features in your original dataframe to be `object` 
We will be treating these as categorical variables.

Change the datatype for each feature in your main dataframe.

In [ ]:
for col in houseprice_df_cleaning.columns:
    if col in object_list:
        houseprice_df_cleaning[col] = houseprice_df_cleaning[col].astype('object')

### Update your meta-info dataframe

In [ ]:
feature_info_df2 = houseprice_df_cleaning.copy()
feature_info_2 = []
for col in feature_info_df2.columns:
    info = { 
        'feature': col,
        'n_unique': len(set(feature_info_df2[col])),
        'datatype': feature_info_df2[col].dtype
    }
    feature_info_2.append(info)
feature_info_df2 = pd.DataFrame(feature_info_2)  
feature_info_df2.head()

### How many columns of each data type are there?

In [ ]:
data_type={}
for col in houseprice_df_cleaning.columns:
    key = houseprice_df_cleaning[col].dtype
    if key in data_type:
        data_type[key] += 1
    else: 
        data_type[key] = 1
data_type     

### Change the remaining integer columns to float columns

In [ ]:
for col in houseprice_df_cleaning.columns:
    if houseprice_df_cleaning[col].dtype =='int64':
        houseprice_df_cleaning[col]= houseprice_df_cleaning[col].astype('float64')

### How many columns of each data type are there?

In [ ]:
data_type={}
for col in houseprice_df_cleaning.columns:
    key = houseprice_df_cleaning[col].dtype
    if key in data_type:
        data_type[key] += 1
    else: 
        data_type[key] = 1
data_type    

### Create numerical and categorical dataframes

Display the shapes of these dataframes.

In [ ]:
num_df = houseprice_df_cleaning.copy()
cat_df = houseprice_df_cleaning.copy()
for col in num_df.columns:
    if num_df[col].dtype == 'O':
        num_df=num_df.drop(col, axis = 1)
        
for col in cat_df.columns:
    if cat_df[col].dtype == 'float64':
        cat_df=cat_df.drop(col, axis = 1)    

### Find the features with nans in the numerical dataframe

In [ ]:
num_null_list = []
for col in num_df.columns:
    if sum(num_df[col].isnull()) > 0:
        num_null_list.append(col)
        print(col,sum(num_df[col].isnull()))        

### Replace the null rows for these features with either the mean or the median

In [ ]:
for col in num_null_list:
    print('mean: ',num_df[col].mean())
    print('median: ', num_df[col].median())
    num_df[col] = num_df[col].where(num_df[col].isnull() == False,num_df[col].mean())
        

### Use subplots to display distribution plots of all numerical features
Include the mean, median, and mode.

In [ ]:
fig, axes = plt.subplots(6,4)
axes = axes.flatten()
fig.set_size_inches(20, 30)
fig.suptitle('Distribution of Numeric Features')
for i, col in enumerate(num_df.columns):
    feature = num_df[col]
    sns.distplot(feature, label=col, ax=axes[i])
    axes[i].axvline(feature.mean(),linewidth=1)
    axes[i].axvline(feature.median(),linewidth=1, color='r')
    axes[i].axvline(feature.mode().values[0],linewidth=1, color='g')
    plt.legend()

### Use `df.skew` to find the skew of the numerical features

In [ ]:
from scipy.stats import skew, skewtest, normaltest
for col in num_df.columns:
    print(col,' P_Value: ', skewtest(num_df[col]).pvalue)

### Make a list of all features for which the absolute value of the skew is > 0.75

These will need to be deskewed.

In [ ]:
from scipy.stats import skew, skewtest, normaltest
skewed_list = []
for col in num_df.columns:
    if abs(skew(num_df[col])) > 0.75:
        skewed_list.append(col)
print(skewed_list)    

### Find the skew for each of these features if we apply a simple log

Either `np.log(feature)` or `np.log(1+feature)`.

In [ ]:
simple_deskewed_num_df = num_df.copy()
for col in skewed_list:
    simple_deskewed_num_df[col] = np.log(1+ simple_deskewed_num_df[col])
    
skewed_list = {}
for col in simple_deskewed_num_df.columns:
    if abs(skew(simple_deskewed_num_df[col])) > 0.75:
        ##print(col)
        ##print(skew(simple_deskewed_num_df[col]))
        ##print(simple_deskewed_num_df[col])
        skewed_list[col] = skew(simple_deskewed_num_df[col])
        
print(skewed_list)

### (Optional) Find the optimal skew for each feature

In [ ]:
-

In [ ]:
deskewed_df = deskew(houseprice_df)

In [ ]:
deskew_plot(houseprice_df)

### Use either a simple log or an optimal log to deskew the selected numerical features

### Use subplots to display distribution plots of all numerical features
Include the mean, median, and mode.

How do your distributions compare to the previous plots?

In [ ]:
fig, axes = plt.subplots(6,4)
axes = axes.flatten()
fig.set_size_inches(20, 30)
fig.suptitle('Distribution of Numeric Features')
for i, col in enumerate(simple_deskewed_num_df.columns):
    feature = num_df[col]
    sns.distplot(feature, label=col, ax=axes[i])
    axes[i].axvline(feature.mean(),linewidth=1)
    axes[i].axvline(feature.median(),linewidth=1, color='r')
    axes[i].axvline(feature.mode().values[0],linewidth=1, color='g')

### Find the features with nans in the categorical dataframe

In [ ]:
cat_nan_count={}
for col in cat_df.columns:
    if sum(cat_df[col].isnull())> 0: 
        cat_nan_count[col] = sum(cat_df[col].isnull())
   
print(cat_nan_count)    

### How many columns of each data type are there?

In [ ]:
data_type={}
for col in cat_df.columns:
    key = cat_df[col].dtype
    if key in data_type:
        data_type[key] += 1
    else: 
        data_type[key] = 1
data_type 

### Replace the null rows for these features with the mode

In [ ]:
cat_nan_count.keys()

In [ ]:
for col in cat_nan_count.keys():
    cat_df[col] = cat_df[col].where(cat_df[col].isnull()== False, cat_df[col].mode()[0])

In [ ]:
for col in cat_df.columns:
    if any(cat_df[col].isnull()) == True:
        print(col)
print('no_null')        

### How many columns of each data type are there?

In [ ]:
data_type={}
for col in cat_df.columns:
    key = cat_df[col].dtype
    if key in data_type:
        data_type[key] += 1
    else: 
        data_type[key] = 1
data_type 

In [ ]:
cat_df.dtypes.value_counts()

### Cast the whole categorical dataframe as an object

In [ ]:
for col in cat_df.columns:
    cat_df[col] = cat_df[col].astype('object')
    

### Get Dummy Variable Columns for all categorical columns

In [ ]:
cat_df_dummies = pd.get_dummies(cat_df)

### What is the shape of the new categorical dataframe?

In [ ]:
cat_df_dummies.shape

In [ ]:
cat_df_dummies.head()

### Merge the processed numerical and categorical dataframes

In [ ]:
simple_deskewed_num_df.info()

In [ ]:
deskewed_df.head()

In [ ]:
houseprice_df_cleaned_deskewed = simple_deskewed_num_df.merge(cat_df_dummies, left_index=True, right_index=True)
houseprice_df_cleaned_deskewed_alpha = deskewed_df.merge(cat_df_dummies, left_index=True, right_index=True)
houseprice_df_cleaned = num_df.merge(cat_df_dummies, left_index=True, right_index=True)

there is a reason to deskew dummies then scale for lasso

In [ ]:
abs(skew(deskewed_df)) < abs(skew(simple_deskewed_num_df))

### Assign `saleprice` to target and the remaining columns to `features`

In [ ]:
target_deskewed = houseprice_df_cleaned_deskewed['saleprice']
features_deskewed =  houseprice_df_cleaned_deskewed.drop('saleprice', axis = 1)
features_deskewed.shape

In [ ]:
target = houseprice_df_cleaned['saleprice']
features =  houseprice_df_cleaned.drop('saleprice', axis = 1)
features.shape

In [ ]:
target_deskewed_scaled_w_Alpha = houseprice_df_cleaned_deskewed_alpha['saleprice']
features_deskewed_scaled_w_Alpha =  houseprice_df_cleaned_deskewed_alpha.drop('saleprice', axis = 1)
display(features_deskewed_scaled_w_Alpha.shape)

pickled_features_deskewed_scaled_w_Alpha = pd.DataFrame.to_pickle(features_deskewed_scaled_w_Alpha,'../pickled/pickled_features_deskewed_scaled_w_Alpha')
pickled_target_deskewed_scaled_w_Alpha = pd.DataFrame.to_pickle(target_deskewed_scaled_w_Alpha,'../pickled/pickled_target_deskewed_scaled_w_Alpha')

### pickle features and target dataframes

In [ ]:
##turn it into a binary and save into a pickle
pickled_features_deskewed = pd.DataFrame.to_pickle(features_deskewed,'../pickled/pickled_features_deskewed')
pickled_target_deskewed = pd.DataFrame.to_pickle(target_deskewed,'../pickled/pickled_target_deskewed')
pickled_features = pd.DataFrame.to_pickle(features,'../pickled/pickled_features')
pickled_target = pd.DataFrame.to_pickle(target,'../pickled/pickled_target')


In [ ]:
def generate_meta_info_df(df):
    feature_info = []
    for col in df.columns:
        info = { 
            'feature': col,
            'n_unique': len(set(df[col])),
            'datatype': df[col].dtype
        
        }
        feature_info.append(info)
    feature_info_df = pd.DataFrame(feature_info)  
    return feature_info_df
        

In [ ]:
generate_meta_info_df(houseprice_df_cleaned)

# 